In [1]:
import mlrun

In [33]:
!ls ./functions

model_evaluation.py  model_training.py	schema_validation.py


In [44]:
project = mlrun.get_or_create_project(
    name="fasttext-project",
    context="./",
    parameters={
        "default_image": "mlrun/mlrun",
        "requirements_file": "./requirements.txt"
    }
)

> 2024-02-23 15:59:44,204 [info] Project loaded successfully: {'project_name': 'fasttext-project'}
